In [1]:
import requests
from bs4 import BeautifulSoup as BS
import csv

In [2]:
nturl = "https://www.nationaltrust.org.uk/sitemap-place.xml"

nt_page = requests.get(nturl).text

In [3]:
soup = BS(nt_page, "xml")

In [4]:
data_list = soup.find_all("xhtml:link")

In [5]:
alinks = [i.get("href") for i in data_list]

In [36]:
for i in alinks[:10]:
#     print(i)
#     place_page = requests.get(i).text
#     place_soup = BS(place_page,"html")
#     place_price = place_soup.find_all("a",{"id": "nt.prices.signpost"})
#     print(i)
#     print(place_price)
#     if place_soup.find_all("a",{"id": "nt.prices.signpost"})== True:
#         place_price = place_soup.find_all("a",{"id": "nt.prices.signpost"})
#         print(place_price)
#     else:
#         pass
    placeclean = ('/').join(i.split('/')[0:4])
    placeclean +="#Prices"
    print(placeclean)
    place_page = requests.get(placeclean).text
    place_soup=BS(place_page,"html.parser")
    category1 = place_soup.find('span',{"id":"nt.prices.category.1.name"})
    if category1:
        print (category1.text)
        name1 = place_soup.find('td',{"id":"nt.prices.category.normal.1.price.1.name"})
        print(name1.text)
        giftaid1 = place_soup.find('td',{"id":"nt.prices.category.normal.1.price.1.giftaid"})
        print(giftaid1.text)
        standard1 = place_soup.find('td',{"id":"nt.prices.category.normal.1.price.1.standard"})
        print(standard1.text)
    else:
        continue
    
  

https://www.nationaltrust.org.uk/segontium#Prices
https://www.nationaltrust.org.uk/cy_gb#Prices
https://www.nationaltrust.org.uk/clyston-mill#Prices
Whole property
Adult

N/A


£5.80

https://www.nationaltrust.org.uk/mendip-hills#Prices
https://www.nationaltrust.org.uk/ulverscroft-nature-reserve#Prices
https://www.nationaltrust.org.uk/white-cliffs-dover#Prices
Car parking charge (per car)
Adult

N/A


£4.00

https://www.nationaltrust.org.uk/bransdale#Prices
https://www.nationaltrust.org.uk/allen-banks-and-staward-gorge#Prices
https://www.nationaltrust.org.uk/sutton-house#Prices
Whole property
Adult

£7.00


£6.30

https://www.nationaltrust.org.uk/duffield-castle#Prices


In [7]:
test_url = "https://www.nationaltrust.org.uk/allen-banks-and-staward-gorge/whats-on"
('/').join(test_url.split('/')[0:4])

'https://www.nationaltrust.org.uk/allen-banks-and-staward-gorge'